In [180]:
import numpy as np
import pandas as pd
import scipy as sc

import random

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [11]:
data_init = pd.read_excel("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/total4.xlsx")

In [19]:
cian = pd.read_csv("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/cian_prices", header=None, sep=";", 
                   names=["sub_area", "avp", "growth"])

In [31]:
cian.head()

sub_area     avp  growth
0       Ajeroport  221317     0.9
1  Akademicheskoe  213748     0.4
2    Alekseevskoe  213848     0.6
3    Altuf'evskoe  151306    -1.3
4           Arbat  526173     7.2

In [30]:
cian.avp = cian.avp.apply(lambda x: int(str(x).replace(" ", "")))

In [283]:
data = data_init.copy()
data["price_m"] = data.price_doc / data.full_sq
data["month"] = [int((d[:4])+(d[5:7])) for d in data.timestamp.values]
data["year"] = [int(d[:4]) for d in data.timestamp.values]
data["expl"] = -data.build_year + data.year

In [284]:
to_take = ["id", "full_sq", "life_sq", "floor", "max_floor", "build_year", "num_room",
          "state", "sub_area", 'kitch_sq','metro_min_avto',"month", "material", "price_doc", "price_m",
          "year", "expl", "kindergarten_km", "school_km", "public_transport_station_km"]

In [285]:
df = data[to_take].reset_index(drop=True)

In [286]:
df = df.merge(cian, on="sub_area")

In [289]:
df["coef"] = df.avp / np.unique(df.avp).mean()
df["pma"] = df.price_m / df.coef

In [290]:
df.head()

id  full_sq  life_sq  floor  max_floor  build_year  num_room  state  \
0    1     43.0     27.0    4.0        9.0      1974.0       2.0    2.0   
1    8     44.0      NaN    5.0        9.0      1974.0       2.0    NaN   
2   33     38.0     34.0    2.0        NaN         NaN       NaN    NaN   
3   71     32.0     19.0    9.0        9.0      1975.0       1.0    2.0   
4  103     44.0     28.0    4.0        9.0      1974.0       2.0    NaN   

   sub_area  kitch_sq      ...              price_m  year  expl  \
0  Bibirevo       6.0      ...        136046.511628  2011  37.0   
1  Bibirevo       6.0      ...         45454.545455  2011  37.0   
2  Bibirevo       NaN      ...        128947.368421  2011   NaN   
3  Bibirevo       6.0      ...        165625.000000  2011  36.0   
4  Bibirevo       6.0      ...        135227.272727  2011  37.0   

   kindergarten_km  school_km  public_transport_station_km     avp  growth  \
0         0.145700   0.177975                     0.274985  154149    -1.6   
1         0.134365   0.581511                     0.254814  154149    -1.6   
2         0.151205   0.295021                     0.054883  154149    -1.6   
3         0.121817   0.346549                     0.167543  154149    -1.6   
4         0.061154   0.595054                     0.227483  154149    -1.6   

       coef            pma  
0  0.864001  157460.989574  
1  0.864001   52609.343836  
2  0.864001  149244.401724  
3  0.864001  191695.296602  
4  0.864001  156512.797912  

[5 rows x 24 columns]

In [291]:
df2 = df.copy()
df2.fillna(0, inplace=True)
df2 = df2[df2.price_doc > 0].reset_index(drop=True)
df3 = df2.drop(["id", "sub_area", "price_doc", "price_m", "growth","month", "coef", "pma"],1)

In [292]:
ss = StandardScaler()
X = ss.fit_transform(df3)

In [293]:
def find_similar(ind):
    v = X[ind]
    r = []
    for i in range(len(X)):
        if i != ind:
            r.append((i,sc.linalg.norm(X[i]-v)))
    r.sort(key=lambda x: x[1], reverse=False)
    rr = []
    for i in range(10):
        rr.append(df2[["pma"]].loc[r[i][0]].values[0])
    rr.sort()
    
    best, std = 0, np.std(rr[:5])
    for i in range(1,len(rr)-5):
        if np.std(rr[i:i+5]) < std:
            best = i
            std = np.std(rr[i:i+5])

    return np.median(np.array(rr[best:best+5]))*df2[["coef"]].loc[ind].values[0]

In [252]:
def find_similar(v):
    r = []
    for i in range(len(X)):
        r.append((i,sc.linalg.norm(X[i]-v)))
    r.sort(key=lambda x: x[1], reverse=False)
    rr = []
    for i in range(10):
        rr.append(df2[["price_m"]].loc[r[i][0]].values[0])
    rr.sort()
    
    best, std = 0, np.std(rr[:5])
    for i in range(1,len(rr)-5):
        if np.std(rr[i:i+5]) < std:
            best = i
            std = np.std(rr[i:i+5])

    return np.median(np.array(rr[best:best+5]))

In [294]:
df3.head()

full_sq  life_sq  floor  max_floor  build_year  num_room  state  kitch_sq  \
0     43.0     27.0    4.0        9.0      1974.0       2.0    2.0       6.0   
1     44.0      0.0    5.0        9.0      1974.0       2.0    0.0       6.0   
2     38.0     34.0    2.0        0.0         0.0       0.0    0.0       0.0   
3     32.0     19.0    9.0        9.0      1975.0       1.0    2.0       6.0   
4     44.0     28.0    4.0        9.0      1974.0       2.0    0.0       6.0   

   metro_min_avto  material  year  expl  kindergarten_km  school_km  \
0        2.590241       1.0  2011  37.0         0.145700   0.177975   
1        3.623270       1.0  2011  37.0         0.134365   0.581511   
2        5.262578       0.0  2011   0.0         0.151205   0.295021   
3        3.914421       1.0  2011  36.0         0.121817   0.346549   
4        3.739761       1.0  2011  37.0         0.061154   0.595054   

   public_transport_station_km     avp  
0                     0.274985  154149  
1                     0.254814  154149  
2                     0.054883  154149  
3                     0.167543  154149  
4                     0.227483  154149

In [233]:
wrong = []

for i in range(len(df2)):
    if abs(df2.price_m.values[i] / df2.avp.values[i] - 1) > 0.4:
        wrong.append(1)
    else:
        wrong.append(0)
df2["wrong"] = wrong

In [182]:
inds = random.sample(list(df2[(df2.state > 0)&(df2.build_year > 0)&(df2.max_floor > 0)&(df2.wrong == 0)
                              &(df2.floor > 0)].index), 1500)

In [295]:
%%time

p1, p2 = [], []

for ind in inds:
    p1.append(df2[["price_m"]].loc[ind].values[0])
    p2.append(find_similar(ind))

Wall time: 17min 4s


In [296]:
l = (np.log1p(np.array(p1)) - np.log1p(np.array(p2)))**2
np.sqrt((1/len(p1))*sum(l))

0.17953140324228103

In [242]:
0.19443164110479511

0.19948650567017986

In [ ]:
df4 = df.copy()
df4.fillna(0, inplace=True)
df4 = df4[df4.price_doc == 0].reset_index(drop=True)
df5 = df4.drop(["id", "sub_area", "price_doc", "price_m", "growth","month"],1)
test_inds = df4.id.values
ss = StandardScaler()
X0 = ss.fit_transform(df5)

In [ ]:
%%time

p = []

for j in range(len(X0)):
    v = X0[j]
    p.append(find_similar(v))

In [ ]:
X0[:,0] * 

In [ ]:
res = pd.DataFrame({"id": test_inds, "price_doc": })